This tutorial is all about maps, see these resources for more of an introduction to data visualizations and making maps:

- [Fundamentals of Data Visualization](https://clauswilke.com/dataviz/)
- [Getting started with Cartopy](https://scitools.org.uk/cartopy/docs/latest/getting_started/index.html)
- [Matplotlib tutorial](https://matplotlib.org/stable/tutorials/index)

If you start seeing a lot of warnings when running the code in this notebook, you may run the cell below to silence them.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Making Maps

In the last notebook, we created a large dataset of temperatures in national parks that we need to search. 

This notebook takes a journey through some plotting just to give you some examples of several kinds of geographic plots you can use. It's mostly for educational purposes, but we can also use it to help find Boreas!

Two commonly used pacakges for plotting in python are [matplotlib](https://matplotlib.org/) (for any kind of data) and [cartopy](https://scitools.org.uk/cartopy/docs/latest/index.html) (an extension to matplotlib that allows for nice geographic plots).

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd
import numpy as np

To start, let's make a plot of the earth and explore some common terms you may need to plot geographic data. The cell below will plot the entire earth using a map [projection](https://scitools.org.uk/cartopy/docs/latest/reference/projections.html) known as the Plate Carree projection.

In [ ]:
fig, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()))

ax.gridlines(draw_labels=True, alpha=0.2, color='black')
ax.stock_img()

## Datums

This section is just for extra information; you don't need it for the rest of the tutorial. Please feel free to skip it if you don't care about the intricacies of plotting geographic data.

Suppose your points show up in weird spots on the map. In that case, two usual suspects are at play: either your map's projection isn't right for your data, or your data's datum doesn't match the projection's datum.

What's a [datum](https://en.wikipedia.org/wiki/Geodetic_datum)? It's basically the "reference frame" that defines how the coordinates relate to the real world. Different datums can make the exact latitude and longitude point to slightly different physical places on Earth. So, if your data and map aren't speaking the same "datum language," your points might be in the wrong spot.

By default, Matplotlib and Cartopy assume your data coordinates use the [WGS84](https://en.wikipedia.org/wiki/World_Geodetic_System) datum (the standard GPS reference) when plotting geographic data. However, suppose your data is based on a different datum, like [NAD27](https://en.wikipedia.org/wiki/North_American_Datum). Simply plotting it without converting it can cause your points to appear shifted or misplaced.

The example below shows how to explicitly transform coordinates from WGS84 (below this is selected using an [epsg](https://en.wikipedia.org/wiki/EPSG_Geodetic_Parameter_Dataset) code [4326](https://epsg.io/4326)) to NAD27 (epsg code [4267](https://epsg.io/4267)) using Python's [pyproj library](https://pyproj4.github.io/pyproj/stable/). It then plots both sets of points on the same map using Cartopy's default PlateCarree projection (which assumes WGS84) so you can see the difference in location caused by the datum shift. Even though the latitude and longitude values may look similar, the actual points correspond to slightly different places on Earth.

Don't worry too much about it now; know these two things can cause headaches when plotting locations. 

In [ ]:
from pyproj import Transformer

# NYC
lon_wgs84, lat_wgs84 = -74.0060, 40.7128

# Transform WGS84 to NAD83
transformer = Transformer.from_crs("EPSG:4326", "EPSG:4267", always_xy=True)
lon_nad83, lat_nad83 = transformer.transform(lon_wgs84, lat_wgs84)

print(f"Difference in degrees: Δlon = {lon_nad83 - lon_wgs84:.8f}, Δlat = {lat_nad83 - lat_wgs84:.8f}")

proj = ccrs.PlateCarree()
fig, ax = plt.subplots(dpi=300, subplot_kw=dict(projection=proj))

ax.set_title("WGS84 vs NAD27 Datum Comparison (New York City)")
ax.stock_img()
ax.coastlines()
ax.gridlines(draw_labels=True, alpha=0.3)

# Plot original WGS84 point
ax.plot(lon_wgs84, lat_wgs84, 'ro', markersize=8, label="WGS84", transform=proj)
ax.text(lon_wgs84 + 0.0001, lat_wgs84 + 0.0005, "WGS84", color='red', transform=proj)

# Plot transformed NAD83 point
ax.plot(lon_nad83, lat_nad83, 'bo', markersize=8, label="NAD83", transform=proj)
ax.text(lon_nad83 + 0.0001, lat_nad83 - 0.0005, "NAD27", color='blue', transform=proj)

# Zoom in to see difference
ax.set_extent([-74.008, -73.998, 40.71, 40.715], crs=proj)

plt.legend()
plt.tight_layout()
plt.show()

## Projections, Web Map Tile Services, Styling

There are many types of projections. All projectons represent the earth in different coordinate systems and can create maps to emphasize different features. The [Plate Caree](https://scitools.org.uk/cartopy/docs/latest/reference/projections.html#platecarree) projection is nice because it represents a 2d grid as if all of the latitude and longitudes are equidistant, meaning that one degree in the latitudue or longitude direciton is equal at all points of the earth (which is not true).

A more common projection you may have heard of is the [Robinson](https://scitools.org.uk/cartopy/docs/latest/reference/projections.html#robinson) projection. To change a map's projection, simply use a different one in the `subplot_kw` dictionary when making a figure.

In [ ]:
fig, ax = plt.subplots(subplot_kw=dict(projection=ccrs.Robinson()))

ax.gridlines(draw_labels=True, alpha=0.2, color='black')
ax.stock_img()

Projections can also be centered around a specific area. For example, we will create a map of the United States using a projection that is often used because it gives the contiguous US a shield shape. To do so, we use the [Albers Equal Area](https://scitools.org.uk/cartopy/docs/latest/reference/projections.html#albersequalarea) projection, using the center of the United States as its focal point. But this time we will also restrict the bounds to a box which bounds the contiguous US. We will also use some higher resolution basemap features since the default `stock_img` would be too pixelated at this smaller scale. These come installed with cartopy in the [feature](https://scitools.org.uk/cartopy/docs/latest/reference/feature.html) subpackage.

You will also notice that when we set the extent (bounding box) of the map that we add a `PlateCaree()` argument. This is not required here because `PlateCaree` and `AlbersEqualArea` both operate on latitude and longitude values, but some prejections operate on a different set of underlying data values. The second argument to `set_extent` specifies the coordiante reference system of the input arguments and allows cartopy to accurately transform the input data points into the coordinate reference system that the map is plotted on. In general it is good practice to specify the coordinate reference system each time you can so that the base map projection can be changed without updating operations on the map.

In [ ]:
fig, ax = plt.subplots(subplot_kw=dict(projection=ccrs.AlbersEqualArea(central_latitude=39.8283, central_longitude=-98.5795)))

minlat = 25
maxlat = 49
minlon = -120
maxlon = -73

ax.set_extent([minlon, maxlon, minlat, maxlat], ccrs.PlateCarree())

ax.gridlines(draw_labels=True, alpha=0.2, color='black')

ax.add_feature(cfeature.LAND.with_scale('10m'))
ax.add_feature(cfeature.OCEAN.with_scale('10m'))
ax.add_feature(cfeature.COASTLINE.with_scale('10m'))
ax.add_feature(cfeature.BORDERS.with_scale('10m'))
ax.add_feature(cfeature.LAKES.with_scale('10m'))
ax.add_feature(cfeature.RIVERS.with_scale('10m'))

That map is a little boring and frankly the colors aren't very fun. We can customize this a bit more to make it more appealing.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6), subplot_kw=dict(
    projection=ccrs.AlbersEqualArea(central_latitude=39.8283, central_longitude=-98.5795)))


ax.gridlines(draw_labels=True, alpha=0.2, color='gray', linestyle='--')
ax.set_extent([minlon, maxlon, minlat, maxlat], ccrs.PlateCarree())

# Custom styling for the land and ocean features
land = cfeature.NaturalEarthFeature('physical', 'land', '10m',
                                    edgecolor='face',
                                    facecolor='#f0e5d8')  # soft tan
ocean = cfeature.NaturalEarthFeature('physical', 'ocean', '10m',
                                     edgecolor='face',
                                     facecolor='#d0e7f2')  # soft blue

ax.add_feature(ocean, zorder=0)
ax.add_feature(land, zorder=1)

ax.add_feature(cfeature.COASTLINE.with_scale('10m'), linewidth=0.5)
ax.add_feature(cfeature.BORDERS.with_scale('10m'), linestyle='--', linewidth=0.6)

ax.add_feature(cfeature.STATES.with_scale('10m'), linewidth=0.4, edgecolor='gray')

ax.add_feature(cfeature.LAKES.with_scale('10m'), facecolor='#a6cee3', edgecolor='k', linewidth=0.2)
ax.add_feature(cfeature.RIVERS.with_scale('10m'), edgecolor='#1f78b4', linewidth=0.4)

ax.set_title("United States Map (Styled)", fontsize=14)
plt.tight_layout()

Well that's cool, but wouldn't it be nice if we could use some real images as the background?

To do so, we can use a [Web Map Tiliing service](https://en.wikipedia.org/wiki/Web_Map_Tile_Service) provided by NASA. We referenced [this example](https://scitools.org.uk/cartopy/docs/latest/gallery/web_services/wmts_time.html#sphx-glr-gallery-web-services-wmts-time-py) to make the image below. 

Below we have chosen to plot September 10, 2020. In this image, we can clearly see the California wildfires that were raging in 2020 at one of the most intense points of the fire.

The cell below may take some time to finish plotting.

In [ ]:
from owslib.wmts import WebMapTileService

plot_crs = ccrs.AlbersEqualArea(central_latitude=39.8283, central_longitude=-98.5795)
fig, ax = plt.subplots(dpi=150, subplot_kw=dict(projection=plot_crs))

url = 'https://gibs.earthdata.nasa.gov/wmts/epsg4326/best/wmts.cgi'
wmts = WebMapTileService(url)

# Layer for VIIRS (Suomi NPP) True Color
layer = 'VIIRS_SNPP_CorrectedReflectance_TrueColor'

date_str = '2020-09-10'

ax.add_wmts(wmts, layer, wmts_kwargs={'time': date_str})

ax.set_extent([minlon, maxlon, minlat, maxlat], ccrs.PlateCarree())
ax.gridlines(draw_labels=True, alpha=0.2, color='black')

Lastly, if you wanted a map with roads and locations on it, you can always use Open Street Maps a a tiler.

In [ ]:
import cartopy.io.img_tiles as cimgt

tiler = cimgt.OSM() 
crs = tiler.crs 

fig, ax = plt.subplots(subplot_kw=dict(projection=crs))

ax.add_image(tiler, 6)  # 12 is the zoom level; adjust for detail level

ax.set_extent([minlon, maxlon, minlat, maxlat], ccrs.PlateCarree())
ax.gridlines(draw_labels=True, alpha=0.2, color='black')

ax.set_title("Open Street Maps Tiling")

## Plotting environmental data on a map

In your time at NCAR, you may have the need to plot some geographic data on top of a map. Below is an example of plotting a heat map of temperatore over the US.

We will elide over the finer points of using xarray right now, but a more complete tutorial can be found [here](https://tutorial.xarray.dev/overview/xarray-in-45-min.html).

Below we load an example dataset from xarray that contains air temperature. It's a 3D dataset (lat, lon, time) with one variable, air temperature. Here, we select the first time step and plot its temperature on the map.

Notice that when we plot the air temperature, we have to supply the `transform` argument. This allows cartopy to convert the lat/lon from the temperature dataset from a Plate Caree projection into the Albers Equal Area projection.

We will also plot points on the map. This may help you find Boreas!

In [ ]:
data = ds = xr.tutorial.load_dataset("air_temperature")

In [ ]:
data

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6), subplot_kw=dict(
    projection=ccrs.AlbersEqualArea(central_latitude=39.8283, central_longitude=-98.5795)))

ax.set_extent([minlon, maxlon, minlat, maxlat], ccrs.PlateCarree())
ax.gridlines(draw_labels=True, alpha=0.2, color='gray', linestyle='--')


ax.add_feature(cfeature.COASTLINE.with_scale('10m'), linewidth=0.5)
ax.add_feature(cfeature.BORDERS.with_scale('10m'), linestyle='--', linewidth=0.6)
ax.add_feature(cfeature.STATES.with_scale('10m'), linewidth=0.4, edgecolor='gray')

ax.spines[:].set_visible(False)

data.isel(time=0)['air'].plot(ax=ax, transform=ccrs.PlateCarree())

lons = data.lon.values
lats = [(maxlat + minlat) / 2] * len(lons)
ax.scatter(lons, lats, transform=ccrs.PlateCarree(), marker='*', color='pink', s=400, edgecolor=None)

plt.title("Air Temperature", fontsize=14)
plt.tight_layout()

# The Clue

Ok, now that we've taken a huge detour from the main mystery with an overview of plotting, let's get back to our investigation!

We will start with plotting the points to see what our data looks like. First, we need to load in the dataset. If you forgot how, 

In [ ]:
filename = "national_park_temperatures.nc"
with xr.open_dataset(filename) as ds:
    ds.load()

To make plotting some things easier, we have also provided you with a csv file of the National Park locations. It's not strictly requried, but you may find a use for it.

In [ ]:
df = pd.read_csv('national_park_locations.csv')

In [ ]:
df

In [ ]:
ds

Next you'll want to make a figure choosing some projections you like. To help you choose a bounds, you can get the minimum and maximum of the coordinates. We've provided that code for you. The rest is up to you! You need to create a scatter plot of the data

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8), dpi=200, subplot_kw=dict(projection=ccrs.PlateCarree()))

ax.gridlines(draw_labels=True, alpha=0.2, color='black')
ax.stock_img()

ax.scatter(df.Longitude.values, df.Latitude.values, color='red', s=10, edgecolor=None, transform=ccrs.PlateCarree())

for index, row in df.iterrows():
    ax.text(row.Longitude, row.Latitude, row.Name, color='blue', transform=ccrs.PlateCarree(), ha='center', va='bottom')

plt.title("Locations: The Mystery of Boreas the Penguin", fontsize=16)

Hmmm. All of the data is clustered in the middle of the US. And from the investigation of `ds` above, we think this is likely data in Boulder, Colorado. Let's zoom in on Boulder and plot again.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8), dpi=200, subplot_kw=dict(projection=ccrs.PlateCarree()))

ax.gridlines(draw_labels=True, alpha=0.2, color='black')
ax.stock_img()

minlat, maxlat = df.Latitude.min() - 1, df.Latitude.max() + 1
minlon, maxlon = df.Longitude.min() - 1, df.Longitude.max() + 1

ax.set_extent([minlon, maxlon, minlat, maxlat], ccrs.PlateCarree())

ax.scatter(df.Longitude.values, df.Latitude.values, color='red', s=10, edgecolor=None, transform=ccrs.PlateCarree())

for index, row in df.iterrows():
    ax.text(row.Longitude, row.Latitude, row.Name, color='blue', transform=ccrs.PlateCarree(), ha='center', va='bottom', fontsize=6)

plt.title("Locations: The Mystery of Boreas the Penguin", fontsize=16)

Now we need to find the minimum temperature of all national parks

In [ ]:
ds.temperature.min()

Alright, we know what the minimum temperature is, but we don't know when or where it occurs. What we need is an index into the latitude array, the longitude array, and the time array that points us to the minimum of air temperature in both space and time.

Fortuntately for us, xarray can tell us where that is using the [argmin](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.argmin.html) function of a dataarray. We can also pass all of our coordinates to the `dim` argument to say we want to know where and when the argument which provides the minimum is.

In [ ]:
ds.temperature.argmin(dim=("lat", "lon", "time"))

This returns a dictionary with three items, the indices for each minimum.

The location of the park with the minimum temperature is at index 3 of the latitude array, index 3 of the longitude array, and index 1634 of the time array. We can use that information to look up the temperature (which we know from above should be 236.29999), and the location and time use the [isel](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.isel.html) function. This function lets us select a value by index.

For the time below, we need to format it as a date string. The time is represetned as a [numpy datetime64[ns]](https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.datetime64), which represnts the number of nanoseconds since the Unix epoch which is 1970-01-01T00:00:00 UTC. To format a number of seconds, you can use the python [strftime](https://docs.python.org/3.6/library/datetime.html#datetime.date.strftime) function to convert this to a date string.

In [ ]:
idxs = ds.temperature.argmin(dim=("lat", "lon", "time"))

lat_i = idxs['lat'].item()
lon_i = idxs['lon'].item()
time_i = idxs['time'].item()

temp = ds.temperature.isel(lat=lat_i, lon=lon_i, time=time_i).item()
coldest_lat = ds.lat[lat_i].item()
coldest_lon = ds.lon[lon_i].item()
time = pd.to_datetime(ds.time[time_i].item()).strftime("%Y-%m-%d %H:%M:%S")
coldest_park = ds.park_name.isel(lat=lat_i, lon=lon_i).item()

# Output
print(f"Minimum temperature: {temp} °K")
print(f"Time: {time}")
print(f"Location: lat={coldest_lat}, lon={coldest_lon}")
print(f"Park name: {coldest_park}")

We've found Boreas! Let's plot a path that Boreas may have taken to get from the NCAR Mesa Lab to the coldest national park. 

In [ ]:
import matplotlib.image as mpimg
import matplotlib.patheffects as PathEffects

mesa_lat, mesa_lon = 39.97806942121865, 254.72514408382477

central_lon = (mesa_lon + coldest_lon) / 2
central_lat = (mesa_lat + coldest_lat) / 2
proj=ccrs.NearsidePerspective(central_longitude=central_lon, central_latitude=central_lat)

# increase the number of points on a line segment between one degree to make the drawn line segment smoother
proj.threshold = proj.threshold/100

fig, ax = plt.subplots(dpi=200, subplot_kw=dict(projection=proj))

ax.gridlines(draw_labels=True, alpha=0.2, color='black')

land = cfeature.NaturalEarthFeature('physical', 'land', '10m',
                                    edgecolor='face',
                                    facecolor='#f0e5d8')  # soft tan
ocean = cfeature.NaturalEarthFeature('physical', 'ocean', '10m',
                                     edgecolor='face',
                                     facecolor='#d0e7f2')  # soft blue

img = mpimg.imread('blue_marble.jpg')

ax.imshow(img, origin='upper',
          transform=ccrs.PlateCarree(),
          extent=[-180, 180, -90, 90],
          zorder=0)

ax.scatter([coldest_lon, mesa_lon], [coldest_lat, mesa_lat], color='red', transform=ccrs.PlateCarree())
ax.plot([coldest_lon, mesa_lon], [coldest_lat, mesa_lat], color='red', transform=ccrs.PlateCarree())

ax.text(
    mesa_lon, mesa_lat, 'Mesa Lab',
    color='white',
    transform=ccrs.PlateCarree(),
    ha='center', va='bottom', fontsize=12,
    path_effects=[
        PathEffects.withStroke(linewidth=1, foreground='black')
    ]
)

ax.text(
    coldest_lon, coldest_lat, coldest_park.replace('-', ' '),
    color='white',
    transform=ccrs.PlateCarree(),
    ha='center', va='bottom', fontsize=12,
    path_effects=[
        PathEffects.withStroke(linewidth=1, foreground='black')
    ]
)

ax.set_title("We found Boreas!", fontsize=16)